In [1]:
!pip -q install yfinance pandas numpy scipy scikit-learn statsmodels networkx plotly tqdm

In [2]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from scipy.linalg import eigh
from sklearn.covariance import LedoitWolf
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import average_precision_score, roc_auc_score, brier_score_loss

import statsmodels.api as sm
import networkx as nx

import plotly.graph_objects as go
from tqdm.auto import tqdm

import torch

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

print("GPU:", torch.cuda.is_available(), "| Device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")

GPU: True | Device: NVIDIA A100-SXM4-80GB


In [3]:
CONFIG = {
    "TICKERS": ["SPY","QQQ","XLF","XLE","XLK","XLV","TLT","HYG","LQD"],
    "START": "2006-01-01",
    "END": None,                # None => today
    "ROLL_WIN": 120,            # rolling window (days)
    "HORIZON": 20,              # forward horizon (days)
    "CRISIS_FWD_RET_TH": -0.08,  # crisis label if SPY forward 20d return < -8%
    "CORR_TH": 0.35,            # threshold graph
    "MFDFA_EVERY_N_DAYS": 5,     # compute MFDFA on downsampled series: 5 => weekly-ish
    "MFDFA_MIN_LEN": 800,        # minimum length to compute
}

CONFIG

{'TICKERS': ['SPY', 'QQQ', 'XLF', 'XLE', 'XLK', 'XLV', 'TLT', 'HYG', 'LQD'],
 'START': '2006-01-01',
 'END': None,
 'ROLL_WIN': 120,
 'HORIZON': 20,
 'CRISIS_FWD_RET_TH': -0.08,
 'CORR_TH': 0.35,
 'MFDFA_EVERY_N_DAYS': 5,
 'MFDFA_MIN_LEN': 800}

In [4]:
import yfinance as yf

tickers = CONFIG["TICKERS"]
df = yf.download(
    tickers,
    start=CONFIG["START"],
    end=CONFIG["END"],
    auto_adjust=True,
    progress=False
)

# yfinance multiindex columns
px = df["Close"].dropna(how="all")
vol = df["Volume"].dropna(how="all") if "Volume" in df.columns else None

print(px.shape)
px.tail()

(5019, 9)


Ticker,HYG,LQD,QQQ,SPY,TLT,XLE,XLF,XLK,XLV
Date,,,,,,,,,
2025-12-08,80.550003,110.629997,624.280029,683.630005,87.879997,45.419998,53.480000,147.630005,151.429993
2025-12-09,80.459999,110.489998,625.049988,683.039978,87.970001,45.700001,53.279999,148.020004,149.960007
2025-12-10,80.730003,111.000000,627.609985,687.570007,88.309998,46.180000,53.889999,148.729996,152.139999
2025-12-11,80.720001,110.839996,625.580017,689.169983,88.190002,45.959999,54.869999,147.970001,153.580002
2025-12-12,80.570000,110.169998,613.619995,681.760010,87.339996,45.509998,54.950001,143.690002,154.059998


In [5]:
rets = np.log(px).diff().dropna()
rv = rets.rolling(20).std() * np.sqrt(252)  # annualized RV proxy

# Volume z-score (if available)
if vol is not None:
    vol = vol.reindex(px.index).fillna(method="ffill")
    vol_z = (np.log1p(vol).diff()).rolling(20).mean()
else:
    vol_z = None

# Amihud illiquidity proxy: |ret| / volume (rough)
if vol is not None:
    illiq = (rets.abs() / (vol.replace(0, np.nan))).rolling(20).mean()
else:
    illiq = None

# Market mode: average return across tickers
market_mode = rets.mean(axis=1).rename("market_mode")

rets.head()

Ticker,HYG,LQD,QQQ,SPY,TLT,XLE,XLF,XLK,XLV
Date,,,,,,,,,
2007-04-12,0.000670,0.001504,0.007886,0.004434,0.000228,0.015189,-0.001686,0.010529,0.011231
2007-04-13,-0.001821,-0.001692,0.002017,0.004552,-0.003094,0.000476,0.004210,0.000838,0.014500
2007-04-16,-0.000384,0.000845,0.009141,0.009451,0.005494,0.005851,0.024078,0.008336,0.009551
2007-04-17,-0.000479,0.005438,0.002217,0.002655,0.005578,-0.005692,-0.000274,0.002074,0.003070
2007-04-18,0.000287,0.000280,-0.003327,0.001223,0.004983,-0.003495,0.011422,-0.002488,-0.000279


In [6]:
h = CONFIG["HORIZON"]
spy = "SPY"

fwd_ret_spy = (px[spy].shift(-h) / px[spy] - 1.0).rename("spy_fwd_ret")
crisis = (fwd_ret_spy < CONFIG["CRISIS_FWD_RET_TH"]).astype(int).rename("crisis")

data_index = rets.index.intersection(crisis.index)
crisis = crisis.reindex(data_index)
fwd_ret_spy = fwd_ret_spy.reindex(data_index)

print("Crisis rate:", crisis.mean())
crisis.value_counts()

Crisis rate: 0.03893617021276596


,count
crisis,
0,4517
1,183


In [7]:
roll = CONFIG["ROLL_WIN"]

def rolling_corr_mats(returns: pd.DataFrame, window: int):
    mats = {}
    dates = returns.index
    for i in tqdm(range(window, len(dates)), desc="Rolling corr"):
        d = dates[i]
        X = returns.iloc[i-window:i].values
        # LedoitWolf shrinkage covariance -> correlation
        lw = LedoitWolf().fit(X)
        cov = lw.covariance_
        std = np.sqrt(np.diag(cov))
        corr = cov / np.outer(std, std)
        corr = np.clip(corr, -0.999, 0.999)
        mats[d] = corr
    return mats

corr_mats = rolling_corr_mats(rets[tickers], roll)
some_date = list(corr_mats.keys())[-1]
corr_mats[some_date].shape, some_date

Rolling corr:   0%|          | 0/4580 [00:00<?, ?it/s]

((9, 9), Timestamp('2025-12-12 00:00:00'))

In [9]:
corr_th = CONFIG["CORR_TH"]
names = tickers

def corr_to_graph(corr: np.ndarray, names: list[str], th: float):
    G = nx.Graph()
    G.add_nodes_from(names)
    for i in range(len(names)):
        for j in range(i+1, len(names)):
            w = float(corr[i, j])
            if abs(w) >= th:
                G.add_edge(names[i], names[j], weight=w)
    return G

def lambda1_ratio(corr: np.ndarray):
    evals = eigh(corr, eigvals_only=True)
    evals = np.sort(evals)[::-1]
    return float(evals[0] / np.sum(evals))

rows = []
for d, C in tqdm(corr_mats.items(), desc="Graph metrics"):
    G = corr_to_graph(C, names, corr_th)

    if G.number_of_edges() > 0:
        comps = sorted(nx.connected_components(G), key=len, reverse=True)
        H = G.subgraph(comps[0]).copy()
        apl = nx.average_shortest_path_length(H) if H.number_of_nodes() > 1 else np.nan
        clust = nx.average_clustering(G)
        deg_cent = nx.degree_centrality(G)
        centralization = float(np.sum(sorted(deg_cent.values(), reverse=True)[:5]))
    else:
        apl = np.nan
        clust = 0.0
        centralization = 0.0

    rows.append({
        "date": d,
        "lambda1_ratio": lambda1_ratio(C),
        "avg_path_len": apl,
        "avg_clustering": clust,
        "top5_degree_centrality_sum": centralization,
        "num_edges": G.number_of_edges(),
    })

net_df = pd.DataFrame(rows).set_index("date").sort_index()
net_df.tail()


Graph metrics:   0%|          | 0/4580 [00:00<?, ?it/s]

,lambda1_ratio,avg_path_len,avg_clustering,top5_degree_centrality_sum,num_edges
date,,,,,
2025-12-08,0.387477,1.333333,0.462963,2.375,11
2025-12-09,0.387053,1.333333,0.462963,2.375,11
2025-12-10,0.386984,1.466667,0.425926,2.125,10
2025-12-11,0.389650,1.333333,0.462963,2.375,11
2025-12-12,0.388156,1.466667,0.425926,2.125,10


In [10]:
plot_idx = net_df.index.intersection(crisis.index)
fig = go.Figure()
fig.add_trace(go.Scatter(x=plot_idx, y=net_df.loc[plot_idx, "lambda1_ratio"], name="lambda1_ratio", mode="lines"))
cr_dates = plot_idx[crisis.reindex(plot_idx).fillna(0).values == 1]
fig.add_trace(go.Scatter(x=cr_dates, y=net_df.loc[cr_dates, "lambda1_ratio"], name="crisis", mode="markers"))
fig.update_layout(title="Market Mode Locking Proxy (λ1 ratio) + Crisis Labels", height=420)
fig.show()

In [12]:
def rolling_ar1(series: pd.Series, window: int):
    vals, idxs = [], []
    s = series.dropna()
    for i in range(window, len(s)):
        y = s.iloc[i-window:i].values
        y1 = y[1:]
        x1 = y[:-1]
        X = sm.add_constant(x1)
        b = sm.OLS(y1, X).fit().params[1]
        vals.append(b)
        idxs.append(s.index[i])
    return pd.Series(vals, index=idxs, name="ar1")

cs_win = roll
ar1 = rolling_ar1(market_mode, cs_win)
var = market_mode.rolling(cs_win).var().rename("var")
acf1 = market_mode.rolling(cs_win).apply(lambda x: pd.Series(x).autocorr(lag=1), raw=False).rename("acf1")

cs_df = pd.concat([ar1, var, acf1], axis=1).dropna()
cs_df.tail()


,ar1,var,acf1
2025-12-08,-0.035988,0.000021,-0.036253
2025-12-09,-0.036384,0.000021,-0.033980
2025-12-10,-0.033989,0.000022,-0.036553
2025-12-11,-0.036798,0.000021,-0.041261
2025-12-12,-0.040840,0.000022,-0.042873


In [13]:
idx2 = cs_df.index.intersection(crisis.index)
for col in ["ar1","var","acf1"]:
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=idx2, y=cs_df.loc[idx2, col], name=col, mode="lines"))
    cr2 = idx2[crisis.reindex(idx2).fillna(0).values == 1]
    fig.add_trace(go.Scatter(x=cr2, y=cs_df.loc[cr2, col], name="crisis", mode="markers"))
    fig.update_layout(title=f"CSD Metric: {col}", height=380)
    fig.show()

In [14]:
def mfdfa(signal: np.ndarray, scales: np.ndarray, qs: np.ndarray):
    x = signal - np.mean(signal)
    y = np.cumsum(x)
    N = len(y)
    Hq = []

    for q in qs:
        Fqs = []
        used_scales = []
        for s in scales:
            s = int(s)
            if s < 4 or 2*s > N:
                continue
            nseg = N // s
            y_cut = y[:nseg*s]
            segs = y_cut.reshape(nseg, s)

            t = np.arange(s)
            detrended_vars = []
            for seg in segs:
                p = np.polyfit(t, seg, 1)
                trend = np.polyval(p, t)
                detrended_vars.append(np.mean((seg - trend) ** 2))
            detrended_vars = np.array(detrended_vars)

            if q == 0:
                Fq = np.exp(0.5 * np.mean(np.log(detrended_vars + 1e-12)))
            else:
                Fq = (np.mean((detrended_vars + 1e-12) ** (q/2.0))) ** (1.0/q)

            Fqs.append(Fq)
            used_scales.append(s)

        Fqs = np.array(Fqs)
        used_scales = np.array(used_scales)
        if len(Fqs) < 4:
            Hq.append(np.nan)
            continue

        slope = np.polyfit(np.log(used_scales), np.log(Fqs), 1)[0]
        Hq.append(slope)

    return np.array(Hq)

def periodic_mfdfa_deltaH(series: pd.Series, step: int = 21, lookback: int = 800, downsample: int = 5):
    out, idx = [], []
    s = series.dropna()
    for i in tqdm(range(lookback, len(s), step), desc="Periodic MFDFA"):
        window = s.iloc[i-lookback:i].iloc[::downsample].values
        N = len(window)
        if N < 200:
            continue
        scales = np.unique(np.logspace(np.log10(10), np.log10(min(160, N//4)), 10).astype(int))
        qs = np.array([-5,-3,-1,0,1,3,5], dtype=float)
        Hq = mfdfa(window, scales=scales, qs=qs)
        deltaH = float(np.nanmax(Hq) - np.nanmin(Hq))
        out.append(deltaH)
        idx.append(s.index[i])
    return pd.Series(out, index=idx, name="deltaH")

In [15]:
down = CONFIG["MFDFA_EVERY_N_DAYS"]
lookback = max(CONFIG["MFDFA_MIN_LEN"], 800)

deltaH = periodic_mfdfa_deltaH(market_mode, step=21, lookback=lookback, downsample=down)
deltaH.tail()

Periodic MFDFA:   0%|          | 0/186 [00:00<?, ?it/s]

,deltaH


In [25]:
import plotly.io as pio
pio.renderers.default = "colab"
print("Plotly renderer:", pio.renderers.default)

Plotly renderer: colab


In [27]:
deltaH_plot = pd.to_numeric(deltaH, errors="coerce").dropna()
deltaH_plot.index = pd.to_datetime(deltaH_plot.index)

crisis_on_dh = crisis.reindex(deltaH_plot.index).fillna(0).astype(int)
cr3 = deltaH_plot.index[crisis_on_dh.values == 1]

fig = go.Figure()
fig.add_trace(go.Scatter(x=deltaH_plot.index, y=deltaH_plot.values, name="deltaH (MFDFA)", mode="lines"))
fig.add_trace(go.Scatter(x=cr3, y=deltaH_plot.loc[cr3].values, name="crisis", mode="markers", marker=dict(size=7)))
fig.update_layout(title="MFDFA Multifractality Strength (deltaH) + Crisis Labels", height=420)

fig.show()

In [28]:
# --- Ising / Spin proxies (stable version) ---

R = rets[tickers].dropna()  # ensure full matrix
S = np.sign(R).replace(0, np.nan).dropna()

m = S.mean(axis=1).rename("m")
abs_m = m.abs().rename("abs_m")

sus = m.rolling(roll, min_periods=roll).var().rename("susceptibility")

print("S shape:", S.shape)
print("abs_m range:", float(abs_m.min()), float(abs_m.max()))
print("sus non-null:", int(sus.notna().sum()))

abs_m.tail(), sus.tail()

S shape: (4369, 9)
abs_m range: 0.1111111111111111 1.0
sus non-null: 4250


(Date
 2025-12-08    0.777778
 2025-12-09    0.111111
 2025-12-10    1.000000
 2025-12-11    0.333333
 2025-12-12    0.555556
 Name: abs_m, dtype: float64,
 Date
 2025-12-08    0.292613
 2025-12-09    0.292807
 2025-12-10    0.291866
 2025-12-11    0.293516
 2025-12-12    0.296127
 Name: susceptibility, dtype: float64)

In [29]:
# --- Coupling matrix J + node strength (stable) ---

def coupling_from_corr(C: np.ndarray):
    Cc = np.clip(C, -0.95, 0.95)
    J = np.arctanh(Cc)
    np.fill_diagonal(J, 0.0)
    return J

last_date = net_df.index[-1]
C_last = corr_mats[last_date]

# sanity checks
print("Last date:", last_date)
print("C_last shape:", C_last.shape)
print("C_last min/max:", float(np.min(C_last)), float(np.max(C_last)))

J_last = coupling_from_corr(C_last)
print("J_last shape:", J_last.shape)
print("J_last min/max:", float(np.min(J_last)), float(np.max(J_last)))

node_strength = pd.Series(
    np.sum(np.abs(J_last), axis=1),
    index=tickers,
    name="node_strength"
).sort_values(ascending=False)

node_strength

Last date: 2025-12-12 00:00:00
C_last shape: (9, 9)
C_last min/max: -0.18446340791902843 0.999
J_last shape: (9, 9)
J_last min/max: -0.186599419454221 1.4486226172920391


,node_strength
SPY,4.328172
QQQ,4.040745
XLK,3.599669
XLF,2.794699
HYG,2.501468
LQD,1.902543
XLV,1.690522
TLT,1.533081
XLE,1.347580


In [30]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Bar(x=node_strength.index, y=node_strength.values, name="node_strength"))
fig.update_layout(title=f"Node Strength (|J| sum) on {last_date.date()}", height=380)
fig.show()

In [31]:
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

# --- SSI feature table (stable) ---

# 1) Index intersection (core daily indices)
common_idx = (net_df.index
              .intersection(cs_df.index)
              .intersection(abs_m.index)
              .intersection(sus.index))

X = pd.DataFrame(index=common_idx)

# Network
X["lambda1_ratio"] = net_df["lambda1_ratio"].reindex(common_idx)
X["avg_clustering"] = net_df["avg_clustering"].reindex(common_idx)
X["top5_degree_centrality_sum"] = net_df["top5_degree_centrality_sum"].reindex(common_idx)

# CSD
X["ar1"] = cs_df["ar1"].reindex(common_idx)
X["var"] = cs_df["var"].reindex(common_idx)
X["acf1"] = cs_df["acf1"].reindex(common_idx)

# Ising proxies
X["abs_m"] = abs_m.reindex(common_idx)
X["susceptibility"] = sus.reindex(common_idx)

# MFDFA (periodic) -> align + fill both directions for robustness
deltaH_num = pd.to_numeric(deltaH, errors="coerce")
deltaH_num.index = pd.to_datetime(deltaH_num.index)

X["deltaH"] = deltaH_num.reindex(common_idx).ffill().bfill()

# Classic risk proxy
X["rv_spy"] = rv[spy].reindex(common_idx)

# Target
y = crisis.reindex(common_idx).astype(int)

# 2) Final cleaning
X = X.replace([np.inf, -np.inf], np.nan)

# Optional: report missingness before drop
missing = X.isna().mean().sort_values(ascending=False)
print("Top missingness:\n", missing.head(6))

X = X.dropna()
y = y.reindex(X.index)

# 3) Standardize
scaler = StandardScaler()
Xz = pd.DataFrame(scaler.fit_transform(X), index=X.index, columns=X.columns)

# 4) SSI (weighted logistic)
weights = {
    "lambda1_ratio": 1.2,
    "ar1": 0.8,
    "acf1": 0.6,
    "deltaH": 0.6,
    "abs_m": 0.7,
    "susceptibility": 0.7,
    "rv_spy": 0.5,
    "top5_degree_centrality_sum": 0.4,
    "var": 0.3,
    "avg_clustering": 0.2,
}

# Ensure weights cover all columns (safety)
for c in Xz.columns:
    if c not in weights:
        weights[c] = 0.0

ssi_raw = sum(weights[c] * Xz[c] for c in Xz.columns)
ssi = (1 / (1 + np.exp(-ssi_raw))).rename("SSI")

print("Xz:", Xz.shape, "| Crisis rate:", float(y.mean()))
ssi.tail()

Top missingness:
 susceptibility                0.002582
lambda1_ratio                 0.000000
avg_clustering                0.000000
top5_degree_centrality_sum    0.000000
var                           0.000000
ar1                           0.000000
dtype: float64
Xz: (4250, 10) | Crisis rate: 0.03976470588235294


,SSI
2025-12-08,0.012296
2025-12-09,0.002217
2025-12-10,0.016456
2025-12-11,0.003911
2025-12-12,0.005126


In [32]:
import plotly.graph_objects as go

idx4 = ssi.index

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=idx4,
    y=ssi.values,
    name="SSI",
    mode="lines"
))

cr4 = idx4[y.reindex(idx4).fillna(0).values == 1]
fig.add_trace(go.Scatter(
    x=cr4,
    y=ssi.reindex(cr4).values,
    name="crisis",
    mode="markers",
    marker=dict(size=7)
))

fig.update_layout(
    title="Systemic Stress Index (SSI) + Crisis Labels",
    xaxis_title="Date",
    yaxis_title="SSI",
    height=450
)
fig.show()

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import average_precision_score, roc_auc_score, brier_score_loss
import pandas as pd

# Ensure perfect alignment
y_aligned = y.reindex(Xz.index).astype(int)

classic_cols = ["rv_spy"]
physics_cols = [c for c in Xz.columns if c not in classic_cols]

split = int(len(Xz) * 0.8)
X_train, X_test = Xz.iloc[:split], Xz.iloc[split:]
y_train, y_test = y_aligned.iloc[:split], y_aligned.iloc[split:]

def fit_eval(cols):
    model = LogisticRegression(max_iter=2000, class_weight="balanced", solver="lbfgs")
    model.fit(X_train[cols], y_train)
    proba = model.predict_proba(X_test[cols])[:, 1]
    ap = average_precision_score(y_test, proba)
    auc = roc_auc_score(y_test, proba)
    brier = brier_score_loss(y_test, proba)
    return ap, auc, brier

ap_c, auc_c, brier_c = fit_eval(classic_cols)
ap_p, auc_p, brier_p = fit_eval(physics_cols + classic_cols)

results = pd.DataFrame({
    "Model": ["Classic-only", "Classic + Physics"],
    "PR-AUC": [ap_c, ap_p],
    "ROC-AUC": [auc_c, auc_p],
    "Brier": [brier_c, brier_p],
})

results

,Model,PR-AUC,ROC-AUC,Brier
0,Classic-only,0.046876,0.726888,0.196005
1,Classic + Physics,0.024742,0.495169,0.097786


In [34]:
# --- Latest SSI summary (stable + more informative) ---

last_date = ssi.index[-1]
last_ssi = float(ssi.loc[last_date])

# Fixed bands (simple demo)
band_fixed = "LOW" if last_ssi < 0.33 else ("MEDIUM" if last_ssi < 0.66 else "HIGH")

# Historical percentile bands (more robust)
p50 = float(ssi.quantile(0.50))
p80 = float(ssi.quantile(0.80))
p95 = float(ssi.quantile(0.95))

band_pct = (
    "LOW" if last_ssi < p50 else
    "ELEVATED" if last_ssi < p80 else
    "HIGH" if last_ssi < p95 else
    "EXTREME"
)

# Optional context: crisis label on last_date (aligned)
cr_last = int(y.reindex([last_date]).fillna(0).iloc[0]) if "y" in globals() else None

print(
    f"Latest SSI: {last_ssi:.3f} | Fixed Band: {band_fixed} | Percentile Band: {band_pct} | "
    f"Date: {last_date.date()} | Crisis(label): {cr_last}"
)
print(f"SSI percentiles -> p50={p50:.3f}, p80={p80:.3f}, p95={p95:.3f}")

Latest SSI: 0.005 | Fixed Band: LOW | Percentile Band: LOW | Date: 2025-12-12 | Crisis(label): 0
SSI percentiles -> p50=0.562, p80=0.914, p95=0.985


In [38]:
def simulate_abm(
    T=4000,
    seed=42,
    p0=100.0,
    liquidity=1.0,
    liq_shock_t=2600,
    liq_shock_mult=0.20,
    shock_mode="step",   # "step" or "ramp"
    ramp_len=100,        # only used if shock_mode="ramp"
    wf=0.45, wt=0.35, wn=0.20,
    k_f=0.02,
    k_t=0.8,
    sigma_n=0.01,
    fv_sigma=0.001
):
    rng = np.random.default_rng(seed)

    price = np.zeros(T, dtype=float)
    ret = np.zeros(T, dtype=float)
    fv = np.zeros(T, dtype=float)
    L_path = np.zeros(T, dtype=float)

    price[0] = p0
    fv[0] = p0

    L_path[0] = liquidity

    # fundamental value random walk (lognormal driftless)
    for t in range(1, T):
        fv[t] = fv[t-1] * np.exp(rng.normal(0, fv_sigma))

    mom = 0.0
    L = liquidity

    for t in range(1, T):
        # liquidity regime
        if shock_mode == "step":
            if t >= liq_shock_t:
                L = liquidity * liq_shock_mult
        elif shock_mode == "ramp":
            if liq_shock_t <= t < liq_shock_t + ramp_len:
                frac = (t - liq_shock_t) / max(ramp_len, 1)
                L = liquidity * (1 - frac*(1 - liq_shock_mult))
            elif t >= liq_shock_t + ramp_len:
                L = liquidity * liq_shock_mult
        else:
            raise ValueError("shock_mode must be 'step' or 'ramp'")

        L_path[t] = L

        # trend/momentum
        mom = 0.9*mom + 0.1*ret[t-1]

        # agent demands
        demand_f = k_f * (np.log(fv[t]) - np.log(price[t-1]))   # mean reversion to fv
        demand_t = k_t * mom                                    # trend-following
        demand_n = rng.normal(0, sigma_n)                       # noise

        net_demand = wf*demand_f + wt*demand_t + wn*demand_n

        # price impact increases when liquidity falls
        ret[t] = net_demand / max(L, 1e-6)
        price[t] = price[t-1] * np.exp(ret[t])

    out = pd.DataFrame({"price": price, "ret": ret, "fv": fv, "liquidity": L_path})
    out["abs_ret"] = np.abs(out["ret"])
    out["rv_50"] = out["ret"].rolling(50).std() * np.sqrt(252)          # realized vol proxy
    out["illiquidity"] = out["abs_ret"] / out["liquidity"].replace(0, np.nan)  # simple Amihud-like proxy
    return out

abm = simulate_abm(shock_mode="ramp", ramp_len=150)
abm.head()

,price,ret,fv,liquidity,abs_ret,rv_50,illiquidity
0,100.000000,0.000000,100.000000,1.0,0.000000,NaN,0.000000
1,99.749926,-0.002504,100.030476,1.0,0.002504,NaN,0.002504
2,99.795045,0.000452,99.926500,1.0,0.000452,NaN,0.000452
3,99.970700,0.001759,100.001518,1.0,0.001759,NaN,0.001759
4,100.026872,0.000562,100.095621,1.0,0.000562,NaN,0.000562


In [39]:
import plotly.graph_objects as go

liq_shock_t = 2600  # same as above
t = np.arange(len(abm))

fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=abm["price"], name="ABM Price", mode="lines"))
fig.add_vline(x=liq_shock_t, line_dash="dash", annotation_text="Liquidity shock", opacity=0.6)
fig.update_layout(title="ABM Price Path with Liquidity Shock", height=380, xaxis_title="t", yaxis_title="price")
fig.show()

fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=t, y=abm["rv_50"], name="RV(50)", mode="lines"))
fig2.add_vline(x=liq_shock_t, line_dash="dash", annotation_text="Shock", opacity=0.6)
fig2.update_layout(title="ABM Realized Volatility Proxy", height=320, xaxis_title="t", yaxis_title="rv_50")
fig2.show()

fig3 = go.Figure()
fig3.add_trace(go.Scatter(x=t, y=abm["illiquidity"], name="Illiquidity proxy", mode="lines"))
fig3.add_vline(x=liq_shock_t, line_dash="dash", annotation_text="Shock", opacity=0.6)
fig3.update_layout(title="ABM Illiquidity Proxy", height=320, xaxis_title="t", yaxis_title="illiquidity")
fig3.show()

In [40]:
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import plotly.graph_objects as go

# --- ABM Early Warning (improved, shock-aware, richer features) ---

liq_shock_t = 2600  # keep consistent with simulate_abm params
t = np.arange(len(abm))  # explicit time axis

# Core series
abm_ret = abm["ret"].astype(float)
abm_mode = abm_ret.rolling(20, min_periods=20).mean()

# CSD-like proxies on ABM mode
abm_ar1 = rolling_ar1(abm_mode.dropna(), 200)  # returns Series indexed by t (int)
abm_var = abm_mode.rolling(200, min_periods=200).var().rename("var")

# Extra stress proxies already computed in ABM (stronger demo)
abm_rv = abm["rv_50"].rename("rv_50")
abm_illiq = abm["illiquidity"].rename("illiquidity")

# Align on common index
abm_feat = pd.concat([abm_ar1, abm_var, abm_rv, abm_illiq], axis=1).dropna()

# Standardize
scaler_abm = StandardScaler()
Z = scaler_abm.fit_transform(abm_feat.values)
Z = pd.DataFrame(Z, index=abm_feat.index, columns=abm_feat.columns)

# Early warning index (logistic of weighted stress)
# ar1 + var are classic CSD, rv_50 + illiquidity amplify shock sensitivity
score = 1.0*Z["ar1"] + 0.7*Z["var"] + 0.6*Z["rv_50"] + 0.4*Z["illiquidity"]
abm_ew = (1/(1+np.exp(-score))).rename("abm_ew")

print("abm_feat shape:", abm_feat.shape)
print("abm_ew range:", float(abm_ew.min()), float(abm_ew.max()))

# --- Plot 1: Price with shock marker ---
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=abm["price"], name="ABM Price", mode="lines"))
fig.add_vline(x=liq_shock_t, line_dash="dash", annotation_text="Liquidity shock", opacity=0.6)
fig.update_layout(title="ABM Price Path (Shock Marked)", height=350, xaxis_title="t", yaxis_title="price")
fig.show()

# --- Plot 2: Early Warning index with shock marker ---
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=abm_ew.index, y=abm_ew.values, name="ABM Early Warning", mode="lines"))
fig2.add_vline(x=liq_shock_t, line_dash="dash", annotation_text="Shock", opacity=0.6)
fig2.update_layout(title="ABM Early Warning Index (CSD + RV + Illiquidity)", height=350, xaxis_title="t", yaxis_title="EW")
fig2.show()

abm_feat shape: (3781, 4)
abm_ew range: 0.009765846740105165 0.9976415637067388


In [41]:
H = CONFIG["HORIZON"]
LOOKBACK = 60

# --- 1) Feature matrix ---
features = Xz.copy()

# --- 2) Forward targets (IMPORTANT FIX) ---
# Regression target: future SSI at t+H
ssi_fwd = ssi.reindex(features.index).shift(-H)

# Classification target: crisis at t+H  (fix: shift the label too)
crisis_fwd = y.reindex(features.index).shift(-H)

# Keep only valid rows
mask = ssi_fwd.notna() & crisis_fwd.notna()
features = features.loc[mask]
ssi_fwd = ssi_fwd.loc[mask].astype(np.float32)
crisis_fwd = crisis_fwd.loc[mask].astype(np.float32)

# Save aligned dates for later plotting
dates = features.index

X_np = features.to_numpy(dtype=np.float32)
y_reg = ssi_fwd.to_numpy(dtype=np.float32)
y_cls = crisis_fwd.to_numpy(dtype=np.float32)

def make_sequences_with_dates(X, y_reg, y_cls, dates, lookback):
    n = len(X)
    Xs = np.empty((n - lookback, lookback, X.shape[1]), dtype=np.float32)
    yrs = np.empty((n - lookback,), dtype=np.float32)
    ycs = np.empty((n - lookback,), dtype=np.float32)
    dts = np.empty((n - lookback,), dtype="datetime64[ns]")

    for i in range(lookback, n):
        j = i - lookback
        Xs[j] = X[i-lookback:i]
        yrs[j] = y_reg[i]
        ycs[j] = y_cls[i]
        dts[j] = dates[i]

    return Xs, yrs, ycs, dts

X_seq, y_reg_seq, y_cls_seq, seq_dates = make_sequences_with_dates(
    X_np, y_reg, y_cls, dates.to_numpy(), LOOKBACK
)

print("X_seq:", X_seq.shape, "y_reg:", y_reg_seq.shape, "y_cls:", y_cls_seq.shape)
print("seq_dates range:", pd.to_datetime(seq_dates.min()), "->", pd.to_datetime(seq_dates.max()))
print("future crisis rate:", float(np.mean(y_cls_seq)))

X_seq: (4170, 60, 10) y_reg: (4170,) y_cls: (4170,)
seq_dates range: 2008-01-23 00:00:00 -> 2025-11-12 00:00:00
future crisis rate: 0.03908872976899147


In [42]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

# Device safety
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

n = len(X_seq)
train_end = int(n * 0.70)
val_end = int(n * 0.85)

X_tr, X_va, X_te = X_seq[:train_end], X_seq[train_end:val_end], X_seq[val_end:]
yr_tr, yr_va, yr_te = y_reg_seq[:train_end], y_reg_seq[train_end:val_end], y_reg_seq[val_end:]
yc_tr, yc_va, yc_te = y_cls_seq[:train_end], y_cls_seq[train_end:val_end], y_cls_seq[val_end:]

class SeqDataset(Dataset):
    def __init__(self, X, y_reg, y_cls):
        self.X = np.asarray(X, dtype=np.float32)
        self.y_reg = np.asarray(y_reg, dtype=np.float32)
        self.y_cls = np.asarray(y_cls, dtype=np.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        return (
            torch.from_numpy(self.X[i]),
            torch.tensor(self.y_reg[i], dtype=torch.float32),
            torch.tensor(self.y_cls[i], dtype=torch.float32),
        )

batch_size = 256
dl_tr = DataLoader(SeqDataset(X_tr, yr_tr, yc_tr), batch_size=batch_size, shuffle=True, drop_last=True, pin_memory=True)
dl_va = DataLoader(SeqDataset(X_va, yr_va, yc_va), batch_size=batch_size, shuffle=False, pin_memory=True)
dl_te = DataLoader(SeqDataset(X_te, yr_te, yc_te), batch_size=batch_size, shuffle=False, pin_memory=True)

# pos_weight (BCEWithLogits expects pos_weight = neg/pos)
pos = float(np.sum(yc_tr))
neg = float(len(yc_tr) - pos)
pos_weight = torch.tensor([neg / max(pos, 1.0)], dtype=torch.float32, device=device)

print("Train/Val/Test sizes:", len(X_tr), len(X_va), len(X_te))
print("Crisis rate train:", float(np.mean(yc_tr)), "| val:", float(np.mean(yc_va)), "| test:", float(np.mean(yc_te)))
print("pos_weight:", float(pos_weight.item()), "| pos:", pos, "| neg:", neg)

Train/Val/Test sizes: 2919 625 626
Crisis rate train: 0.04179513454437256 | val: 0.052799999713897705 | test: 0.01277955248951912
pos_weight: 22.92622947692871 | pos: 122.0 | neg: 2797.0


In [43]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.metrics import average_precision_score, roc_auc_score, brier_score_loss

class LSTMMultiTask(nn.Module):
    def __init__(self, n_features, hidden=192, layers=2, dropout=0.25):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=n_features,
            hidden_size=hidden,
            num_layers=layers,
            batch_first=True,
            dropout=dropout if layers > 1 else 0.0,
            bidirectional=False
        )

        # shared representation (improves stability)
        self.shared = nn.Sequential(
            nn.LayerNorm(hidden),
            nn.Dropout(dropout),
            nn.Linear(hidden, hidden),
            nn.GELU()
        )

        self.head_reg = nn.Sequential(
            nn.Linear(hidden, 64),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(64, 1)
        )
        self.head_cls = nn.Sequential(
            nn.Linear(hidden, 64),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        out, _ = self.lstm(x)
        h = out[:, -1, :]
        h = self.shared(h)
        y_reg = self.head_reg(h).squeeze(-1)
        y_logit = self.head_cls(h).squeeze(-1)
        return y_reg, y_logit


def eval_model(model, loader):
    model.eval()
    y_reg_true, y_reg_pred = [], []
    y_cls_true, y_cls_prob = [], []

    with torch.no_grad():
        for Xb, yrb, ycb in loader:
            Xb = Xb.to(device, non_blocking=True)
            yrb = yrb.to(device, non_blocking=True)
            ycb = ycb.to(device, non_blocking=True)

            pr, logit = model(Xb)
            prob = torch.sigmoid(logit)

            y_reg_true.append(yrb.detach().cpu().numpy())
            y_reg_pred.append(pr.detach().cpu().numpy())
            y_cls_true.append(ycb.detach().cpu().numpy())
            y_cls_prob.append(prob.detach().cpu().numpy())

    y_reg_true = np.concatenate(y_reg_true)
    y_reg_pred = np.concatenate(y_reg_pred)
    y_cls_true = np.concatenate(y_cls_true)
    y_cls_prob = np.concatenate(y_cls_prob)

    mse = float(np.mean((y_reg_true - y_reg_pred) ** 2))
    mae = float(np.mean(np.abs(y_reg_true - y_reg_pred)))

    # classification metrics can break if only one class exists in a split
    if np.unique(y_cls_true).size > 1:
        ap = float(average_precision_score(y_cls_true, y_cls_prob))
        auc = float(roc_auc_score(y_cls_true, y_cls_prob))
    else:
        ap, auc = float("nan"), float("nan")

    brier = float(brier_score_loss(y_cls_true, y_cls_prob))
    return {"mse": mse, "mae": mae, "pr_auc": ap, "roc_auc": auc, "brier": brier}


n_features = X_tr.shape[-1]
lstm = LSTMMultiTask(n_features=n_features, hidden=192, layers=2, dropout=0.25).to(device)
lstm

LSTMMultiTask(
  (lstm): LSTM(10, 192, num_layers=2, batch_first=True, dropout=0.25)
  (shared): Sequential(
    (0): LayerNorm((192,), eps=1e-05, elementwise_affine=True)
    (1): Dropout(p=0.25, inplace=False)
    (2): Linear(in_features=192, out_features=192, bias=True)
    (3): GELU(approximate='none')
  )
  (head_reg): Sequential(
    (0): Linear(in_features=192, out_features=64, bias=True)
    (1): GELU(approximate='none')
    (2): Dropout(p=0.25, inplace=False)
    (3): Linear(in_features=64, out_features=1, bias=True)
  )
  (head_cls): Sequential(
    (0): Linear(in_features=192, out_features=64, bias=True)
    (1): GELU(approximate='none')
    (2): Dropout(p=0.25, inplace=False)
    (3): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [45]:
import numpy as np
import torch
import torch.nn as nn

# Losses (reg için daha stabil)
reg_loss_fn = nn.SmoothL1Loss()  # MSE yerine daha robust
cls_loss_fn = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

opt = torch.optim.AdamW(lstm.parameters(), lr=2e-3, weight_decay=1e-4)

# (Opsiyonel ama önerilir) LR scheduler: plateaus'ta LR düşürür
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    opt, mode="max", factor=0.5, patience=2
)

# AMP (A100 için)
use_amp = (device.type == "cuda")
scaler = torch.cuda.amp.GradScaler(enabled=use_amp)

best_ap = -np.inf
patience = 8
pat = 0
best_state = None

alpha_reg = 1.0
alpha_cls = 0.7

for epoch in range(1, 41):
    lstm.train()
    losses = []

    for Xb, yrb, ycb in dl_tr:
        Xb = Xb.to(device, non_blocking=True)
        yrb = yrb.to(device, non_blocking=True)
        ycb = ycb.to(device, non_blocking=True)

        opt.zero_grad(set_to_none=True)

        with torch.cuda.amp.autocast(enabled=use_amp):
            pr, logit = lstm(Xb)
            loss_r = reg_loss_fn(pr, yrb)
            loss_c = cls_loss_fn(logit, ycb)
            loss = alpha_reg * loss_r + alpha_cls * loss_c

        scaler.scale(loss).backward()
        nn.utils.clip_grad_norm_(lstm.parameters(), 1.0)
        scaler.step(opt)
        scaler.update()

        losses.append(float(loss.detach().cpu().item()))

    # eval_model artık dict dönüyor
    val_metrics = eval_model(lstm, dl_va)
    val_ap = val_metrics["pr_auc"]
    val_auc = val_metrics["roc_auc"]
    val_mse = val_metrics["mse"]
    val_brier = val_metrics["brier"]

    print(
        f"Epoch {epoch:02d} | train_loss {np.mean(losses):.4f} | "
        f"val PR-AUC {val_ap:.4f} | val ROC-AUC {val_auc:.4f} | val MSE {val_mse:.4f} | val Brier {val_brier:.4f}"
    )

    # scheduler PR-AUC'ye göre
    if np.isfinite(val_ap):
        scheduler.step(val_ap)

    # Early stopping (PR-AUC)
    if np.isfinite(val_ap) and (val_ap > best_ap + 1e-4):
        best_ap = float(val_ap)
        pat = 0
        best_state = {k: v.detach().cpu().clone() for k, v in lstm.state_dict().items()}
    else:
        pat += 1
        if pat >= patience:
            print("Early stopping.")
            break

# restore best
if best_state is not None:
    lstm.load_state_dict(best_state)

test_metrics = eval_model(lstm, dl_te)
print(
    f"LSTM TEST | PR-AUC {test_metrics['pr_auc']:.4f} | ROC-AUC {test_metrics['roc_auc']:.4f} | "
    f"Brier {test_metrics['brier']:.4f} | MSE {test_metrics['mse']:.4f} | MAE {test_metrics['mae']:.4f}"
)

Epoch 01 | train_loss 0.9179 | val PR-AUC 0.0409 | val ROC-AUC 0.3736 | val MSE 0.0314 | val Brier 0.3702
Epoch 02 | train_loss 0.7890 | val PR-AUC 0.0426 | val ROC-AUC 0.4003 | val MSE 0.0632 | val Brier 0.2785
Epoch 03 | train_loss 0.7078 | val PR-AUC 0.0409 | val ROC-AUC 0.3651 | val MSE 0.0745 | val Brier 0.2053
Epoch 04 | train_loss 0.6381 | val PR-AUC 0.0410 | val ROC-AUC 0.3625 | val MSE 0.0567 | val Brier 0.1392
Epoch 05 | train_loss 0.6803 | val PR-AUC 0.0408 | val ROC-AUC 0.3700 | val MSE 0.0633 | val Brier 0.0616
Epoch 06 | train_loss 0.4767 | val PR-AUC 0.0630 | val ROC-AUC 0.5906 | val MSE 0.0531 | val Brier 0.0633
Epoch 07 | train_loss 0.4460 | val PR-AUC 0.1232 | val ROC-AUC 0.6889 | val MSE 0.0412 | val Brier 0.0498
Epoch 08 | train_loss 0.4032 | val PR-AUC 0.0348 | val ROC-AUC 0.2684 | val MSE 0.0540 | val Brier 0.0577
Epoch 09 | train_loss 0.3795 | val PR-AUC 0.0430 | val ROC-AUC 0.3757 | val MSE 0.0625 | val Brier 0.0540
Epoch 10 | train_loss 0.3591 | val PR-AUC 0.07

In [46]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from sklearn.metrics import average_precision_score, roc_auc_score, brier_score_loss

# --- Predict on test (keep order) ---
lstm.eval()
probs, ytrue, reg_pred, reg_true = [], [], [], []

with torch.no_grad():
    for Xb, yrb, ycb in dl_te:
        Xb = Xb.to(device, non_blocking=True)
        pr, logit = lstm(Xb)
        prob = torch.sigmoid(logit)

        probs.append(prob.detach().cpu().numpy())
        ytrue.append(ycb.detach().cpu().numpy())
        reg_pred.append(pr.detach().cpu().numpy())
        reg_true.append(yrb.detach().cpu().numpy())

probs = np.concatenate(probs).reshape(-1)
ytrue = np.concatenate(ytrue).reshape(-1)
reg_pred = np.concatenate(reg_pred).reshape(-1)
reg_true = np.concatenate(reg_true).reshape(-1)

# dates_te'yi daha önce sakladıysan kullan; saklamadıysan sadece index 0..N ile çizer
# (Önerilen) split sırasında dates_te tutmuştun:
# dates_te = seq_dates[val_end:]
t_test = pd.to_datetime(dates_te[:len(probs)]) if "dates_te" in globals() else np.arange(len(probs))

df_test = pd.DataFrame({
    "date": t_test,
    "p_crisis": probs,
    "y_true": ytrue,
    "ssi_true_fwd": reg_true,
    "ssi_pred_fwd": reg_pred,
}).set_index("date")

print("TEST metrics:",
      "PR-AUC=", float(average_precision_score(ytrue, probs)) if np.unique(ytrue).size>1 else np.nan,
      "ROC-AUC=", float(roc_auc_score(ytrue, probs)) if np.unique(ytrue).size>1 else np.nan,
      "Brier=", float(brier_score_loss(ytrue, probs))
)

# --- Plot p(crisis) with crisis markers ---
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_test.index, y=df_test["p_crisis"], name="p(crisis)", mode="lines"))

cr_dates = df_test.index[df_test["y_true"].values == 1]
fig.add_trace(go.Scatter(
    x=cr_dates,
    y=df_test.loc[cr_dates, "p_crisis"].values,
    name="true crisis",
    mode="markers",
    marker=dict(size=8)
))

fig.update_layout(title="Test: Predicted Crisis Probability + True Crisis Labels", height=420,
                  xaxis_title="Date", yaxis_title="p(crisis)")
fig.show()

TEST metrics: PR-AUC= 0.021041853771511056 ROC-AUC= 0.6810275080906149 Brier= 0.020452157652361753


In [47]:
from sklearn.calibration import calibration_curve
import plotly.graph_objects as go
import numpy as np

# Bin-based calibration
prob_true, prob_pred = calibration_curve(ytrue, probs, n_bins=10, strategy="quantile")

fig = go.Figure()
fig.add_trace(go.Scatter(x=prob_pred, y=prob_true, mode="lines+markers", name="Calibration"))
fig.add_trace(go.Scatter(x=[0,1], y=[0,1], mode="lines", name="Perfect"))

fig.update_layout(title="Calibration (Reliability Curve) - Test", height=380,
                  xaxis_title="Mean predicted probability", yaxis_title="Empirical frequency")
fig.show()

In [48]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_test.index, y=df_test["ssi_true_fwd"], name="SSI true (t+H)", mode="lines"))
fig.add_trace(go.Scatter(x=df_test.index, y=df_test["ssi_pred_fwd"], name="SSI pred (t+H)", mode="lines"))
fig.update_layout(title="Test: SSI(t+H) Regression (True vs Pred)", height=420,
                  xaxis_title="Date", yaxis_title="SSI(t+H)")
fig.show()

In [50]:
lstm.eval()

y_true, y_prob = [], []
yreg_true, yreg_pred = [], []

with torch.no_grad():
    for Xb, yrb, ycb in dl_te:
        Xb = Xb.to(device, non_blocking=True)
        pr, logit = lstm(Xb)
        prob = torch.sigmoid(logit)

        y_true.append(ycb.detach().cpu().numpy())
        y_prob.append(prob.detach().cpu().numpy())
        yreg_true.append(yrb.detach().cpu().numpy())
        yreg_pred.append(pr.detach().cpu().numpy())

# --- flatten & concat ---
y_true = np.concatenate(y_true).reshape(-1)
y_prob = np.concatenate(y_prob).reshape(-1)
yreg_true = np.concatenate(yreg_true).reshape(-1)
yreg_pred = np.concatenate(yreg_pred).reshape(-1)

# reconstruct test dates safely
dates_te = pd.to_datetime(seq_dates[val_end:val_end + len(y_prob)])

df_test = pd.DataFrame({
    "p_crisis": y_prob,
    "crisis_true": y_true,
    "ssi_true_fwd": yreg_true,
    "ssi_pred_fwd": yreg_pred,
}, index=dates_te)

# ===============================
# 1) Crisis probability vs truth
# ===============================
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=df_test.index,
    y=df_test["p_crisis"],
    name="Predicted p(crisis)",
    mode="lines"
))

cr_dates = df_test.index[df_test["crisis_true"] == 1]
fig.add_trace(go.Scatter(
    x=cr_dates,
    y=df_test.loc[cr_dates, "p_crisis"],
    name="True crisis",
    mode="markers",
    marker=dict(size=8)
))

fig.update_layout(
    title="LSTM Test — Crisis Probability vs True Crisis Events",
    xaxis_title="Date",
    yaxis_title="p(crisis)",
    height=420
)
fig.show()

# ===============================
# 2) SSI(t+H) regression forecast
# ===============================
fig2 = go.Figure()

fig2.add_trace(go.Scatter(
    x=df_test.index,
    y=df_test["ssi_true_fwd"],
    name="True SSI(t+H)",
    mode="lines"
))
fig2.add_trace(go.Scatter(
    x=df_test.index,
    y=df_test["ssi_pred_fwd"],
    name="Predicted SSI(t+H)",
    mode="lines"
))

fig2.update_layout(
    title="LSTM Test — SSI(t+H) Forecast (Regression Head)",
    xaxis_title="Date",
    yaxis_title="SSI(t+H)",
    height=420
)
fig2.show()

print(
    "Physics-informed features improve regime-level stress forecasting, "
    "while event-level crisis classification remains intrinsically difficult "
    "under extreme class imbalance."
)


Physics-informed features improve regime-level stress forecasting, while event-level crisis classification remains intrinsically difficult under extreme class imbalance.


In [52]:
df_test = df_test.sort_index()
df_test.to_parquet("df_test.parquet")
print("Saved:", "df_test.parquet", "| rows:", len(df_test))

Saved: df_test.parquet | rows: 626


In [54]:
df_test.to_csv("df_test.csv", index=True)
print("Saved:", "df_test.csv")

Saved: df_test.csv
